In [10]:
from tensorflow.keras.models import load_model
import geopandas as gps
import rasterio                  # I/O raster data (netcdf, height, geotiff, ...)
import rasterio.warp             # Reproject raster samples
from rasterio import windows
from rasterio import features
import fiona                     # I/O vector data (shape, geojson, ...)
import pyproj                    # Change coordinate reference system
from osgeo import gdal, ogr, osr
import pandas as pd
import shapely
from shapely.geometry import Point, Polygon
from shapely.geometry import mapping, shape
import cv2
import json
import geopandas as gpd
import numpy as np               # numerical array manipulation
import pandas as pd
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from tqdm import tqdm
from PIL import Image
import PIL.ImageDraw

from itertools import product

import sys
from core.UNet import UNet
from core.losses import tversky, focalTversky, bce_dice_loss, accuracy, dice_loss, IoU, recall, precision
from core.optimizers import adaDelta, adagrad, adam, nadam
from core.visualize import display_images
# from core.dataset_generator import DataGenerator

import matplotlib.pyplot as plt  # plotting tools
%matplotlib inline
import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
import tensorflow as tf
print(tf.__version__)

2.5.0-rc3


In [11]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto(
    #device_count={"CPU": 64},
    allow_soft_placement=True, 
    log_device_placement=False)
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [21]:
# Required configurations (including the input and output paths) are stored in a separate file (such as config/RasterAnalysis.py)
# Please provide required info in the file before continuing with this notebook. 
 
from config import RasterAnalysis
# In case you are using a different folder name such as configLargeCluster, then you should import from the respective folder 
# Eg. from configLargeCluster import RasterAnalysis
config = RasterAnalysis.Configuration()

self.trained_model_path: D:\sample250\U-Net_13\saved_models\UNet\lakes_20230818-2031_AdaDelta_dice_loss_b5_012345_512.h5


In [14]:
# Load a pretrained model 
OPTIMIZER = adaDelta
# OPTIMIZER = adam
OPTIMIZER=tf.train.experimental.enable_mixed_precision_graph_rewrite(OPTIMIZER)
model = load_model(config.trained_model_path, custom_objects={'dice loss': dice_loss, 'accuracy':accuracy ,'recall':recall, 'precision':precision,'IoU': IoU}, compile=False)
model.compile(optimizer=OPTIMIZER, loss=dice_loss, metrics=[dice_loss, accuracy,recall, precision, IoU])

In [15]:
# Methods to add results of a patch   to    the total results of a larger area. 
#The operator could be min (useful if there are too many false positives), max (useful for tackle false negatives)
#res:mask [rows,cols] predition=np.squeeze(prediction[i], axis = -1) (col, row, wi, he) = batch_pos[i]
def addTOResult(res, prediction, row, col, he, wi,operator,padding):
    currValue = res[row:row+he, col:col+wi]
    newPredictions = prediction[:he, :wi]
# IMPORTANT: MIN can't be used as long as the mask is initialed with 0!!!!! 
#If you want to use MIN initial the mask with -1 and handle the case of default value(-1) separately.
    if operator == 'min': # Takes the min of current prediction and new prediction for each pixel  
        currValue [currValue == -1] = 1 #Replace -1 with 1 in case of MIN  
        res[row:row+he, col:col+wi] =  np.minimum(currValue, newPredictions)
    elif operator == 'max':
        res[row:row+he, col:col+wi] = np.maximum(currValue, newPredictions)
    elif operator == 'padding':
        newPredictions = newPredictions[padding:he-padding, padding:wi-padding]
        res[row+padding:row+he-padding, col+padding:col+wi-padding] =newPredictions
    else:
        res[row:row+he, col:col+wi] = newPredictions  
    return (res)

In [16]:
def readBands(fn,window):
    img=rasterio.open(fn)
    im=img.read(window=window)/1000
    return im

# Methods that actually makes the predictions
def predict_using_model(model, batch, batch_pos, mask,operator,padding):
#     print(batch)
    tm = np.stack(batch, axis = 0)
#     print(tm)
    prediction = model.predict(tm)
    for i in range(len(batch_pos)): 
        (col, row, wi, he) = batch_pos[i]
        p = np.squeeze(prediction[i], axis = -1)
        # Instead of replacing the current values with new values, use the user specified operator (MIN,MAX,REPLACE)
        mask = addTOResult(mask, p, row, col, he, wi,operator,padding)  
    return mask
    
def detect_lake(fullPath,NDWI_img,operator, width=512, height=512, stride = 256,padding=100):
    nols, nrows = NDWI_img.meta['width'], NDWI_img.meta['height']
    meta = NDWI_img.meta.copy() 
    if 'float' not in meta['dtype']: #The prediction is a float so we keep it as float to be consistent with the prediction. 
        meta['dtype'] = np.float32
    col_index=list(range(0, nols-width, stride))
    col_index.append(nols-width)
    row_index=list(range(0, nrows-height, stride))
    row_index.append(nrows-height)
    offsets = product(col_index,row_index)
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    print('the size of current NDWI_img',nrows, nols) 

    mask = np.zeros((nrows, nols), dtype=meta['dtype'])

#     mask = mask -1   # Note: The initial mask is initialized with -1 instead of zero   to handle the MIN case (see addToResult)
    batch = []
    batch_pos = [ ]
    for col_off, row_off in  tqdm(offsets):
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, NDWI_img.transform) 
#         hbh:notice datatype is float
        patch = np.full((height, width, 5),-1.0)#Add -1 padding in case of corner images
    
        read_NDWI_img=NDWI_img.read(window=window)/100
        read_red_img =readBands(fullPath.replace(config.ndwi_fn_st ,config.red_fn_st),window)
        read_green_img =readBands(fullPath.replace(config.ndwi_fn_st ,config.green_fn_st),window)
        read_blue_img = readBands(fullPath.replace(config.ndwi_fn_st ,config.blue_fn_st),window)
        read_swir_img = readBands(fullPath.replace(config.ndwi_fn_st ,config.swir_fn_st),window)
        temp_im =  np.concatenate((read_NDWI_img,read_red_img,read_green_img,read_blue_img, read_swir_img), axis=0)
        temp_im =  np.transpose(temp_im, axes=(1,2,0))
        patch[:window.height, :window.width] = temp_im   
        
        batch.append(patch)
        batch_pos.append((window.col_off, window.row_off, window.width, window.height))
        if (len(batch) == config.BATCH_SIZE):
            mask = predict_using_model(model, batch, batch_pos, mask,operator,padding)
            batch = []
            batch_pos = []
            
    # To handle the edge of images as the image size may not be divisible by n complete batches and few frames on the edge may be left.
    if batch:
        mask = predict_using_model(model, batch, batch_pos, mask,operator,padding)
        batch = []
        batch_pos = []

    return(mask, meta)

In [17]:
def raster2vector(raster_path, vecter_path, field_name="class", ignore_values = None):
    
    # 读取路径中的栅格数据
    raster = gdal.Open(raster_path)
    # in_band 为想要转为矢量的波段,一般需要进行转矢量的栅格都是单波段分类结果
    # 若栅格为多波段,需要提前转换为单波段
    band = raster.GetRasterBand(1)
    
    # 读取栅格的投影信息,为后面生成的矢量赋予相同的投影信息
    prj = osr.SpatialReference()
    prj.ImportFromWkt(raster.GetProjection())
    
    
    drv = ogr.GetDriverByName("ESRI Shapefile")
    # 若文件已经存在,删除
    if os.path.exists(vecter_path):
        drv.DeleteDataSource(vecter_path)
        
    # 创建目标文件
    polygon = drv.CreateDataSource(vecter_path)
    # 创建面图层
    poly_layer = polygon.CreateLayer(vecter_path[:-4], srs=prj, geom_type=ogr.wkbMultiPolygon)
    # 添加浮点型字段,用来存储栅格的像素值
    field = ogr.FieldDefn(field_name, ogr.OFTReal)  
    poly_layer.CreateField(field)
    
    # FPolygonize将每个像元转成一个矩形，然后将相似的像元进行合并
    # 设置矢量图层中保存像元值的字段序号为0
    gdal.FPolygonize(band, None, poly_layer, 0)
    
    # 删除ignore_value链表中的类别要素
    if ignore_values is not None:
        for feature in poly_layer:
            class_value = feature.GetField('class')
            for ignore_value in ignore_values:
                if class_value==ignore_value:
                    # 通过FID删除要素
                    poly_layer.DeleteFeature(feature.GetFID())
                    break
                
    polygon.SyncToDisk()
    polygon = None
    print('Vector File Exported Successfully!')
        
def writeMaskToDisk(detected_mask, detected_meta, wp, write_as_type = 'uint8', th = 0.5, create_countors = False):
    # Convert to correct required before writing
    if 'float' in str(detected_meta['dtype']) and 'int' in write_as_type:
        print(f'Converting prediction from {detected_meta["dtype"]} to {write_as_type}, using threshold of {th}')#float32 to uint8
#         initial code have problem of big lake
        detected_mask[detected_mask<th]=0
        detected_mask[detected_mask>=th]=1
        detected_mask = detected_mask.astype(write_as_type)#'uint8'
        detected_meta['dtype'] =  write_as_type
    
    # compress tif
    detected_meta.update({"compress": 'lzw'})
    
    with rasterio.open(wp, 'w', **detected_meta) as outds:
        outds.write(detected_mask, 1)

In [22]:
# Predict trees in the all the files in the input image dir 
# Depending upon the available RAM, images may not to be split before running this cell.
# Use the Auxiliary-2-SplitRasterToAnalyse if the images are too big to be analysed in memory.
all_files = []
for root, dirs, files in os.walk(config.input_image_dir):
    for file in files:
#         if file.endswith(config.input_image_type) and file.startswith(config.NDWI_fn_st):
        if file.endswith(config.input_image_type)and file.startswith(config.ndwi_fn_st):
             all_files.append((os.path.join(root, file), file))
print(all_files)

operator='padding'
for fullPath, filename in all_files:
    output = filename.replace(config.input_image_type,config.output_image_type)
    outputFile = os.path.join(config.output_dir, output.replace(config.ndwi_fn_st, config.output_prefix + operator))
    if not os.path.isfile(outputFile) or config.overwrite_analysed_files:
        with rasterio.open(fullPath) as NDWI:
            print(fullPath)
            detectedMask, detectedMeta = detect_lake(fullPath,NDWI,operator,width = config.WIDTH, height = config.HEIGHT, stride = config.STRIDE)
            writeMaskToDisk(detectedMask, detectedMeta, outputFile, write_as_type = config.output_dtype, th = 0.5, create_countors = False)            
    else:
        print('File already analysed!', fullPath)
        
print('finish')

1it [00:00,  6.73it/s]

[('D:\\sample250\\test_png\\ndwi_178.png', 'ndwi_178.png'), ('D:\\sample250\\test_png\\ndwi_24.png', 'ndwi_24.png'), ('D:\\sample250\\test_png\\ndwi_26.png', 'ndwi_26.png'), ('D:\\sample250\\test_png\\ndwi_29.png', 'ndwi_29.png'), ('D:\\sample250\\test_png\\ndwi_35.png', 'ndwi_35.png'), ('D:\\sample250\\test_png\\ndwi_41.png', 'ndwi_41.png'), ('D:\\sample250\\test_png\\ndwi_103.png', 'ndwi_103.png'), ('D:\\sample250\\test_png\\ndwi_152.png', 'ndwi_152.png'), ('D:\\sample250\\test_png\\ndwi_156.png', 'ndwi_156.png'), ('D:\\sample250\\test_png\\ndwi_165.png', 'ndwi_165.png'), ('D:\\sample250\\test_png\\ndwi_168.png', 'ndwi_168.png'), ('D:\\sample250\\test_png\\ndwi_181.png', 'ndwi_181.png'), ('D:\\sample250\\test_png\\ndwi_188.png', 'ndwi_188.png'), ('D:\\sample250\\test_png\\ndwi_196.png', 'ndwi_196.png'), ('D:\\sample250\\test_png\\ndwi_202.png', 'ndwi_202.png'), ('D:\\sample250\\test_png\\ndwi_0.png', 'ndwi_0.png'), ('D:\\sample250\\test_png\\ndwi_15.png', 'ndwi_15.png'), ('D:\\sample

20it [00:04,  4.29it/s]
0it [00:00, ?it/s]

Converting prediction from float32 to uint8, using threshold of 0.5
D:\sample250\test_png\ndwi_24.png
the size of current NDWI_img 926 1624


18it [00:04,  3.78it/s]
0it [00:00, ?it/s]

...

the size of current NDWI_img 1664 1216


24it [00:05,  4.80it/s]
0it [00:00, ?it/s]

Converting prediction from float32 to uint8, using threshold of 0.5
D:\sample250\test_png\ndwi_243.png
the size of current NDWI_img 2679 3570


130it [00:35,  3.70it/s]


Converting prediction from float32 to uint8, using threshold of 0.5
File already analysed! D:\sample250\test_png\test\ndwi_0.png
finish
